# Mistral's Models on Amazon Bedrock

Welcome to our workshop introducing Mistral’s models on Amazon Bedrock. In this workshop we will demonstrate the long context window capabilities of Mistral Large 2.

Let's dive in!


## What is Mistral Large 2?

Mistral Large 2 (24.07) is a state-of-the-art large language model featuring:

- **128 Billion Parameters:** Enhancing its ability to understand and generate complex language structures.
- **128k Context Window:** Allowing it to process and generate responses based on very long inputs.
- **Multilingual Proficiency:** Supporting dozens of languages, including French, German, Spanish, Italian, Arabic, Hindi, Japanese, and more.
- **Coding Language Support:** Understanding and generating code in over 80 programming languages.
- **Improved Instruction Following:** Better adherence to user instructions and tasks.
- **Enhanced Conversational Abilities:** More natural and context-aware interactions.
- **Tool Use:** Ability to utilize tools and functions for extended operations.

## Model Details

* **Available Regions**: `us-west-2`
* **Model ID**: `mistral.mistral-large-2407-v1:0`
* **Context Window**: 128,000 tokens
* **Maximum Tokens per Response**: 8,192

To learn more about Mistral Large 2 benchmarks, follow this [link](https://mistral.ai/news/mistral-large-2407/)

In this notebook, we'll guide you through the process of using Mistral Large 2 to summarize a PDF document, demonstrating its capacity to handle long contexts and generate detailed summaries.


## Importing Necessary Libraries

To interact with Amazon Bedrock and process PDF files, we need to import the following libraries:

* **`boto3`**: AWS SDK for Python, used to interact with Amazon Bedrock.
* **`botocore.config.Config`**: Allows configuration of AWS clients, such as setting timeouts.
* **`PyPDF2`**: A library for reading and extracting text from PDF files.

In [1]:
%pip install -r requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install boto3 botocore PyPDF2 --quiet

Note: you may need to restart the kernel to use updated packages.


In [18]:
import boto3
import logging
import json
from botocore.config import Config
import PyPDF2
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

### Initialize the Bedrock Client

The `initialize_bedrock_client` function sets up the client for interacting with Amazon Bedrock.

### Converse with the Model

The `converse` function sends a prompt to the Mistral Large 2 model and retrieves the response.

### Extract Text from PDF

The `extract_text_from_pdf` function reads a PDF file and extracts all the text content.

In [19]:
# Utility Functions
def initialize_bedrock_client(region_name="us-west-2", read_timeout=2000):
    config = Config(read_timeout=read_timeout)
    return boto3.client(
        service_name='bedrock-runtime',
        region_name=region_name,
        config=config
    )

def converse(
    system_prompt='',
    task_instructions='',
    context='',
    max_tokens=1000,
    temperature=0.1,
    top_p=0.9,
    model_id='mistral.mistral-large-2407-v1:0',
    bedrock_client=None
):
    if bedrock_client is None:
        bedrock_client = initialize_bedrock_client()
    # Construct the system prompt
    system = [{"text": system_prompt}] if system_prompt else []

    # Construct the user message
    user_content = '\n'.join(filter(None, [task_instructions, context]))

    messages = [{
        "role": "user",
        "content": [{"text": user_content.strip()}]
    }]

    try:
        # Make the converse API call
        response = bedrock_client.converse(
            modelId=model_id,
            messages=messages,
            system=system,
            inferenceConfig={
                "maxTokens": max_tokens,
                "temperature": temperature,
                "topP": top_p
            }
        )

        #Input token logger with reponse metadata
        token_usage = response['usage']
        logger.info("Input tokens: %s", token_usage['inputTokens'])
        # Extract and return the assistant's response
        assistant_response = response["output"]["message"]["content"][0]["text"]
        return assistant_response.strip()

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            # Use a generator expression to extract text from all pages
            text = "\n".join(page.extract_text() or "" for page in reader.pages)
            return text
    except FileNotFoundError:
        print(f"Error: The file '{pdf_path}' was not found. Please check the file path.")
    except Exception as e:
        print(f"An error occurred while reading the PDF file: {e}")
    return ""


## Prompting Techniques with Mistral Large 2

Few-Shot Learning, Delimiters, and Role Playing

Few-shot learning or in-context learning is when we provide a few examples in the prompts, and the LLM can generate corresponding output based on these examples. This technique can often improve model performance, especially when the task is difficult or when we want the model to respond in a specific manner.

Delimiters like ###, <<< >>>, or other symbols specify the boundary between different sections of the text. In our examples, we'll use ### to indicate examples and <<< >>> to indicate customer inquiries.

Role playing involves providing the LLM with a role (e.g., "You are a bank customer service bot.", "You are a pirate"), which adds personal context to the model and often leads to better performance - this is best defined within the system field. You can combine the persona definition and the instructions within the system field as well - you may have to experiment to see which approach yields higher performance for your specific use case. 



In [20]:
# Define the system prompt and task instructions directly
system = "You are a business analyst who provides clear summaries of customer feedback, identifying key issues and suggesting actionable improvements."

task_instructions = """
### Example 1:
Customer Feedback: "I've been using your software for a few months now, and while it's generally good, it crashes whenever I try to export reports. This is really frustrating and hinders my work."
Analysis:
- **Issue Identified**: Software crashes during report export.
- **Suggested Improvement**: Fix the bug causing crashes during the export function to enhance user experience.

### Example 2:
Customer Feedback: "The user interface is not intuitive. It took me a long time to find basic features, and the navigation is confusing."
Analysis:
- **Issue Identified**: Unintuitive user interface and confusing navigation.
- **Suggested Improvement**: Redesign the UI to be more user-friendly and streamline navigation menus.

###
Customer Feedback: "Your customer service is unresponsive. I reached out multiple times about an issue, but haven't received any assistance."
Analysis:
"""

# Since we don't need the PDF for this example, we can proceed without context
# Call the converse function
response = converse(
    system_prompt=system,
    task_instructions=task_instructions,
    context="",  # No additional context needed
    max_tokens=300,
    temperature=0.0,
    top_p=0.9
)

# Print the assistant's response
print(response)


INFO:__main__:Input tokens: 249


- **Issue Identified**: Inefficient customer service response time and lack of assistance.
- **Suggested Improvement**: Improve customer service response time by increasing staffing, providing additional training, or implementing a more efficient ticketing system. Also, consider offering multiple channels for support, such as live chat, email, and phone, to better assist customers. Regularly update customers on the status of their issues and provide clear timelines for resolution.


## Information Retrieval in Long Contexts

Language models often struggle to locate information embedded in the middle of long texts due to context window limitations or challenges in attention mechanisms. Mistral Large 2, with its 128k context window, is designed to handle such tasks more effectively (sometimes referred to as the needle in a haystack problem).

Example: extracting specific information from the middle of our pdf document. On page 22, there's an insight we'll ask Large 2 to find: 

***“Amazon AppFlow is a fully managed integration service that enables customers to securely transfer data between Software-as-a-Service (SaaS) applications such as Salesforce, Marketo, Slack, and ServiceNow, and AWS services such as Amazon S3 and Amazon Redshift. AppFlow can run data flows at a frequency the customer chooses - on a schedule, in response to a business event, or on demand.”***



In [26]:
# Define the system prompt
system_prompt = "You are an expert assistant who can find and summarize specific sections of long documents."

# Task instructions
task_instructions = """
Given the AWS Security whitepaper, help me answer this question - if you do not know the answer, say 'I don't know': 

What is Amazon AppFlow, and how does it integrate with SaaS applications and AWS services to facilitate secure data transfer?
"""

# Extract text from the PDF
pdf_path = 'AWS-security-whitepaper.pdf'
document_text = extract_text_from_pdf(pdf_path)

# Call the converse function
response = converse(
    system_prompt=system_prompt,
    task_instructions=task_instructions,
    context=document_text,
    max_tokens=500,
    temperature=0.0,
    top_p=0.9
)

# Print the assistant's response
print(response)


INFO:__main__:Input tokens: 44025


### Amazon AppFlow

**Amazon AppFlow** is a fully managed integration service that enables customers to securely transfer data between Software-as-a-Service (SaaS) applications such as Salesforce, Marketo, Slack, and ServiceNow, and AWS services such as Amazon S3 and Amazon Redshift. AppFlow can run data flows at a frequency the customer chooses—on a schedule, in response to a business event, or on demand. Customers can also configure data transformation capabilities like filtering and validation to generate rich, ready-to-use data as part of the flow itself, without additional steps.

#### Key Features for HIPAA Compliance:

1. **Encryption in Transit**:
   - Amazon AppFlow encrypts data while in transit between AppFlow and the configured source/destination using TLS 1.2 or later.

2. **Encryption at Rest**:
   - Data stored at rest in S3 is automatically encrypted using an AWS KMS key (formerly CMK) that is specified by the customer.
   - For PHI data transferred to non-S3 destinatio

## Summarizing a Lengthy PDF Document with Mistral Large 2

In this section, we'll utilize Mistral Large 2 to generate a comprehensive summary of a lengthy PDF document—the AWS Security whitepaper. This demonstration showcases the model's capability to handle long contexts and produce detailed summaries that capture the essence of the original material. By extracting the text from the PDF and crafting specific prompts, we'll guide the model to generate an organized summary that includes an overview, key insights, challenges, and a concise conclusion.

We'll set up our prompts and execute the summarization. Remember, you can define Mistral Large 2's persona in the system field and specify the task instructions in the user role. Feel free to update the persona to whatever you like and adjust the task instructions to suit your needs. Today, we'll focus on extracting the text from the PDF and preparing the prompts for the summarization.

In [27]:
def summarize_document(pdf_path, system_prompt, task_instructions, max_tokens=1000):
    # Extract text from the PDF
    document_text = extract_text_from_pdf(pdf_path)

    # Check if the document was loaded successfully before proceeding
    if document_text:
        # Call the converse function to summarize the document
        response = converse(
            system_prompt=system_prompt,
            task_instructions=task_instructions,
            context=document_text,
            max_tokens=max_tokens,
            temperature=0.1,
            top_p=0.9
        )
        return response
    else:
        print("Cannot proceed with summarization due to issues with loading the document.")
        return None

In [29]:
# Main Execution
if __name__ == "__main__":
    # Initialize Bedrock client
    bedrock_client = initialize_bedrock_client()

    # Define prompts
    system_prompt = "You are a polite research assistant who is always helpful, cheerful, pragmatic, and extremely detail oriented"
    task_instructions = """
    Please provide a comprehensive summary of the document, including the following sections:
    1. **Overview**
   - A very brief introduction to the main topic and objectives of the paper in about ten sentences.

2. **Key Insights**
   - Detailed insights and findings presented in the paper.
   - Highlight any opportunities identified by the authors.

3. **Key Challenges**
   - Outline the main challenges or obstacles discussed.
   - Discuss any limitations or areas that require further research.

4. **Conclusion**
   - A very concise wrap-up of the overall significance of the findings in a few sentences.
   
Ensure that each section is clearly labeled and that the information is presented in a clear and organized manner.
    """

    # Summarize the document
    pdf_path = 'AWS-security-whitepaper.pdf'
    summary = summarize_document(pdf_path, system_prompt, task_instructions)

    # Print the summarized response
    if summary:
        print("### Summary of the Document ###\n")
        print(summary)


INFO:__main__:Input tokens: 44142


### Summary of the Document ###

### Overview

The AWS Whitepaper "Architecting for HIPAA Security and Compliance on Amazon Web Services" outlines how customers can use AWS to run sensitive workloads regulated under the U.S. Health Insurance Portability and Accountability Act (HIPAA). The paper focuses on the HIPAA Privacy and Security Rules for protecting Protected Health Information (PHI) and how to use AWS to encrypt data in transit and at rest. It also discusses how AWS features can be used to run workloads containing PHI. The document emphasizes that AWS offers a comprehensive set of features and services to manage key encryption and make it simpler to audit, including the AWS Key Management Service (AWS KMS). Customers with HIPAA compliance requirements have flexibility in how they meet encryption requirements for PHI. The paper also highlights that AWS provides a Business Associate Addendum (BAA) for such customers, which is a standardized agreement for processing, storing, and 

### Note:

In this notebook we used the converse api from Bedrock to maximize the 128k context window of Mistral Large 2.

In certain scenarios, you might be dealing with individual documents or document stores that are much larger than the context windows most large language models offer. In these situations, you are able leverage Open-source frameworks like [LangChain](https://www.langchain.com/) with your documents to use techniques such as [`map-reduce`](https://js.langchain.com/v0.1/docs/modules/chains/document/map_reduce/)to chunk them and generate individual summaries quickly in parallel and reduce to a single summary or [`refine`](https://js.langchain.com/v0.1/docs/modules/chains/document/refine/) to iteratively process document chunks by looping over the input documents to generate a summary with better retention of context between chunks.

## Conclusion

In this notebook, we explored the advanced capabilities of Mistral Large 2 on Amazon Bedrock through practical examples. We demonstrated how to effectively use few-shot learning and prompt engineering techniques to guide the model in analyzing customer feedback. We showcased Mistral Large 2's ability to perform information retrieval in long contexts, finding specific information like a "needle in a haystack" within extensive texts. Lastly, we leveraged its powerful summarization capabilities to condense lengthy documents into comprehensive summaries. These examples highlight the model's proficiency in handling complex tasks, making it a valuable tool for a wide range of applications across various industries